In [1]:
!pip install gensim
!pip install python-Levenshtein

  Obtaining dependency information for FuzzyTM>=0.4.0 from https://files.pythonhosted.org/packages/06/4d/8d2dd5d81afdea2aa790860d5c7e12f80154923ba827e3fa36759f0bf2cd/FuzzyTM-2.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for pyfume from https://files.pythonhosted.org/packages/f0/fe/b899a3d9a18c9a44a35155c79a4c152cb85990ea38ce6ab7ed73e5caa1b9/pyFUME-0.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for simpful from https://files.pythonhosted.org/packages/9d/0e/aebc2fb0b0f481994179b2ee2b8e6bbf0894d971594688c018375e7076ea/simpful-2.12.0-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/59.6 kB ? eta -:--:--
   ---------------------------------------- 59.6/59.6 kB 3.1 MB/s eta 0:00:00
  Created wheel for fst-pso: filename=fst_pso

In [9]:
import gensim
import pandas as pd

In [10]:
df = pd.read_json("C:/Users/LENOVO/Downloads/reviews_Cell_Phones_and_Accessories_5.json/Cell_Phones_and_Accessories_5.json",lines = True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [11]:
df.shape

(194439, 9)

# Preprocessing and Tokenization

In [12]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [13]:
gensim.utils.simple_preprocess(df['reviewText'][0])

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [14]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

# Train the Word2Vec Model

In [15]:
model = gensim.models.Word2Vec(window=10, vector_size=100, min_count=2, sg=1)

# window=10: Kích thước của "cửa sổ ngữ cảnh" (context window) mà mô hình sử dụng để học từ các từ xung quanh. 
# Một giá trị window bằng 10 có nghĩa là mô hình sẽ xem xét 10 từ trước và 10 từ sau từ trung tâm để dự đoán từ trung tâm.

# size=100: Số chiều của các vector từ (word embeddings). 
# Trong ví dụ này, mỗi từ sẽ được biểu diễn bằng một vector 100 chiều.

# min_count=2: Đây là số lượng tối thiểu các lần xuất hiện của một từ để nó được đưa vào mô hình. 
# Nếu một từ xuất hiện ít hơn 2 lần, nó sẽ bị loại bỏ khỏi tập dữ liệu đào tạo.

# sg=1: Chỉ định phương thức đào tạo của mô hình. Khi sg được đặt là 1, mô hình sẽ sử dụng kiến trúc Skip-gram; 
# nếu sg=0, mô hình sẽ sử dụng kiến trúc CBOW 

In [16]:
model.build_vocab(review_text) # build the vocabulary

In [17]:
model.epochs

# Một "epoch" trong quá trình huấn luyện mô hình máy học đề cập đến một lần lặp qua toàn bộ tập dữ liệu đào tạo 
# mà thuật toán sử dụng để cập nhật trọng số mô hình.

5

In [18]:
model.window

10

# Train Word2Vec Model

In [21]:
model.train(review_text,total_examples = model.corpus_count,epochs = model.epochs)

(61510711, 83868975)

In [20]:
model.corpus_count

# model.corpus_count cho bạn biết số lượng "tài liệu" hoặc "câu" trong tập dữ liệu mà mô hình đã được đào tạo trên

194439

# Save the Model

In [22]:
model.save("C:/Users/LENOVO/Desktop/final_word2vec.model")

# Find the Similar Words and Similarity between Words

In [25]:
model.wv.most_similar('bad')

[('terrible', 0.6823469400405884),
 ('horrible', 0.6580029726028442),
 ('iits', 0.6541630625724792),
 ('bleached', 0.6374953389167786),
 ('okay', 0.6325086951255798),
 ('expierence', 0.6308104991912842),
 ('ok', 0.6242256760597229),
 ('gonba', 0.6238527297973633),
 ('trilled', 0.6209409832954407),
 ('good', 0.6160014271736145)]

In [27]:
model.wv.similarity(w1='bad',w2='terrible')

0.68234694

In [49]:
matrix = model.wv.vectors   #extract the word embedding matrix
print(f"shape of the word embedding matrix is {matrix.shape}")

shape of the word embedding matrix is (35561, 100)


In [72]:
W_out = model.syn1neg
W_out.shape # Output weights

(35561, 100)

In [99]:
vocab_list = list(model.wv.key_to_index.keys())

In [88]:
len(vocab_list)

35561

In [90]:
vocab_list

['the',
 'it',
 'and',
 'to',
 'is',
 'this',
 'of',
 'for',
 'my',
 'that',
 'in',
 'on',
 'phone',
 'with',
 'you',
 'case',
 'but',
 'have',
 'not',
 'was',
 'as',
 'so',
 'one',
 'very',
 'are',
 'like',
 'if',
 'be',
 'can',
 'or',
 'great',
 'your',
 'at',
 'when',
 'use',
 'screen',
 'just',
 'good',
 'all',
 'they',
 'battery',
 'from',
 'would',
 'out',
 'will',
 'well',
 'an',
 'has',
 'iphone',
 'had',
 'get',
 'charge',
 'up',
 'no',
 'me',
 'than',
 'more',
 'only',
 'charger',
 'about',
 'product',
 'other',
 'there',
 'really',
 'time',
 'also',
 'off',
 'these',
 'which',
 'works',
 'does',
 'because',
 'do',
 'don',
 'them',
 'much',
 'back',
 'what',
 'nice',
 'little',
 'price',
 'love',
 'usb',
 'its',
 'some',
 'quality',
 'charging',
 'work',
 'fit',
 'any',
 'easy',
 'even',
 've',
 'device',
 'too',
 'after',
 'still',
 'used',
 'protector',
 'while',
 'power',
 'using',
 'got',
 'better',
 'am',
 'bought',
 'two',
 'now',
 'by',
 'cable',
 'first',
 'recommend'

In [95]:
vocab_len = len(vocab_list)
vocab = model.wv.index_to_key

In [100]:
for word in model.wv.key_to_index:
    print(f"{word}: {model.wv.get_vecattr(word, 'count')}")


the: 999914
it: 556798
and: 493522
to: 457132
is: 322030
this: 270357
of: 238986
for: 237538
my: 230750
that: 197003
in: 184211
on: 183089
phone: 181782
with: 175733
you: 165260
case: 150067
but: 145913
have: 133853
not: 133410
was: 109917
as: 106110
so: 97154
one: 89962
very: 79169
are: 77189
like: 74082
if: 73869
be: 73863
can: 72110
or: 71750
great: 68907
your: 66568
at: 66106
when: 64390
use: 63250
screen: 62404
just: 61992
good: 60372
all: 60191
they: 59703
battery: 59391
from: 58259
would: 55733
out: 54928
will: 54547
well: 53710
an: 52247
has: 52035
iphone: 51118
had: 49070
get: 47276
charge: 46319
up: 46302
no: 46040
me: 45765
than: 43084
more: 42404
only: 40659
charger: 40160
about: 40138
product: 39677
other: 39222
there: 38878
really: 38749
time: 37991
also: 37875
off: 37317
these: 36547
which: 35804
works: 33675
does: 33622
because: 33207
do: 32516
don: 32497
them: 32207
much: 31830
back: 31463
what: 30719
nice: 30356
little: 30294
price: 30219
love: 29776
usb: 29607
its: 2

In [137]:
for word in model.wv.index_to_key:
    print(f"{word}: {model.wv.get_vecattr(word, 'count')}")

the: 999914
it: 556798
and: 493522
to: 457132
is: 322030
this: 270357
of: 238986
for: 237538
my: 230750
that: 197003
in: 184211
on: 183089
phone: 181782
with: 175733
you: 165260
case: 150067
but: 145913
have: 133853
not: 133410
was: 109917
as: 106110
so: 97154
one: 89962
very: 79169
are: 77189
like: 74082
if: 73869
be: 73863
can: 72110
or: 71750
great: 68907
your: 66568
at: 66106
when: 64390
use: 63250
screen: 62404
just: 61992
good: 60372
all: 60191
they: 59703
battery: 59391
from: 58259
would: 55733
out: 54928
will: 54547
well: 53710
an: 52247
has: 52035
iphone: 51118
had: 49070
get: 47276
charge: 46319
up: 46302
no: 46040
me: 45765
than: 43084
more: 42404
only: 40659
charger: 40160
about: 40138
product: 39677
other: 39222
there: 38878
really: 38749
time: 37991
also: 37875
off: 37317
these: 36547
which: 35804
works: 33675
does: 33622
because: 33207
do: 32516
don: 32497
them: 32207
much: 31830
back: 31463
what: 30719
nice: 30356
little: 30294
price: 30219
love: 29776
usb: 29607
its: 2

In [141]:
model.wv.key_to_index['bad']

272

In [135]:
import numpy as np
one_hot = np.zeros(shape = (1,vocab_len))

# tạo ra ma trận 1 hàng và vocab_len cột toàn các số 0

one_hot[0,model.wv.key_to_index['bad']] = 1 

# vector sẽ có giá trị 0 ở mọi vị trí trừ vị trí đại diện cho từ "bad", nơi mà giá trị sẽ là 1.

In [124]:
z_proj = np.dot(one_hot,matrix) # Nhân ma trận với một hidden layer(ở đây chính là matrix)
z_proj.shape

(1, 100)

In [126]:
a_proj = z_proj
z_out = np.dot(a_proj,W_out.T) # lấy kết quả sau khi nhân với hidden layer đi nhân với trọng số syn1neg thì ra output
z_out.shape

(1, 35561)

In [127]:
#Softmax
a_out = np.exp(z_out)/np.sum(np.exp(z_out)) # Dùng active function với output

In [129]:
a_out.shape

(1, 35561)

In [136]:
a_out = a_out.flatten()

indices = np.argsort(a_out) # sort in ascending order
indices = indices[::-1] # sort in descending order
for i in range(10):
    word_index = indices[i]
    word = vocab_int2word[word_index]
    a_out_i = a_out[word_index]
    print(f"{word} with score {a_out_i}")

bass with score 0.0011744328232581782
sound with score 0.0010420997229330918
quality with score 0.000936965826803109
speaker with score 0.0008590200892773949
highs with score 0.0006971684569839427
build with score 0.0006402240572854356
mids with score 0.0006021158622672247
value with score 0.0005788044092844572
treble with score 0.0005556547704566377
range with score 0.0005542596951827613
